## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Load the dataset

In [2]:
df= pd.read_csv(r"C:\Users\sanoj\Downloads\dynamic_pricing (1).csv")
df.head()

,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride
0,90,45,Urban,Silver,13,4.47,Night,Premium,90,284.257273
1,58,39,Suburban,Silver,72,4.06,Evening,Economy,43,173.874753
2,42,31,Rural,Silver,0,3.99,Afternoon,Premium,76,329.795469
3,89,28,Rural,Regular,67,4.31,Afternoon,Premium,134,470.201232
4,78,22,Rural,Regular,74,3.77,Afternoon,Economy,149,579.681422


In [3]:
df.describe()

,Number_of_Riders,Number_of_Drivers,Number_of_Past_Rides,Average_Ratings,Expected_Ride_Duration,Historical_Cost_of_Ride
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000
mean,60.372000,27.076000,50.031000,4.257220,99.58800,372.502623
std,23.701506,19.068346,29.313774,0.435781,49.16545,187.158756
min,20.000000,5.000000,0.000000,3.500000,10.00000,25.993449
25%,40.000000,11.000000,25.000000,3.870000,59.75000,221.365202
50%,60.000000,22.000000,51.000000,4.270000,102.00000,362.019426
75%,81.000000,38.000000,75.000000,4.632500,143.00000,510.497504
max,100.000000,89.000000,100.000000,5.000000,180.00000,836.116419


# Ratio Based Approach

## Calculating Demand_Supply_Ratio and picking demand_supply_threshold = 2.3 arround the mean of Demand_Supply_Ratio
* ### Higher Demand = when 'Demand_Supply_Ratio' > demand_supply_threshold (2.3) else Low-demand
* ### Higher supply = when 'Demand_Supply_Ratio' < demand_supply_threshold (2.3) else Low-supply



In [4]:
df['Demand_Supply_Ratio'] = df['Number_of_Riders'] / df['Number_of_Drivers']
demand_supply_threshold = 2.3

df['Demand_class'] = np.where(df['Demand_Supply_Ratio'] > demand_supply_threshold, "Higher_demand", "Lower_demand")
df['Supply_class'] = np.where(df['Demand_Supply_Ratio'] < demand_supply_threshold, "Higher_supply", "Lower_supply")

df.iloc[:,[0,1,10,11,12]].sample(10)

,Number_of_Riders,Number_of_Drivers,Demand_Supply_Ratio,Demand_class,Supply_class
214,49,30,1.633333,Lower_demand,Higher_supply
592,33,20,1.650000,Lower_demand,Higher_supply
751,47,24,1.958333,Lower_demand,Higher_supply
610,97,9,10.777778,Higher_demand,Lower_supply
952,26,5,5.200000,Higher_demand,Lower_supply
340,86,10,8.600000,Higher_demand,Lower_supply
845,68,16,4.250000,Higher_demand,Lower_supply
420,40,30,1.333333,Lower_demand,Higher_supply
652,55,29,1.896552,Lower_demand,Higher_supply
156,94,59,1.593220,Lower_demand,Higher_supply


## calulation Base Price and Surge_charge based on supply demand ratio and demand_supply_factor
* ### 1. Calculate base historical cost based on expected_Ride_duration
* ### 2. Calculate rider-to-driver ratio
* ### 3. Calculate demand-supply factor
* ### 4. Defining a methode to Calculate supply_demand_surge and Apply the dynamic pricing formula

In [5]:

constant_rate = 3.5  # Define the base rate per unit of duration, this is arround mean of ratio of ('Historical_Cost_of_Ride'/'Expected_Ride_Duration')
demand_hike = 0.35  # This is how much demand increase the pricing

# Calculate base historical cost based on expected_Ride_duration
df['base_cost'] = df['Expected_Ride_Duration'] * constant_rate

# Calculate rider-to-driver ratio
df['rider_driver_ratio'] = df['Number_of_Riders'] / df['Number_of_Drivers']

# Calculate demand-supply factor
df['demand_supply_factor'] = df['rider_driver_ratio'] - 1
df['demand_supply_factor'] = df['demand_supply_factor'].apply(lambda x: min(x, 6))

# defining a methode to Calculate supply_demand_surge and Apply the dynamic pricing formula
def apply_surge(df):
    SD_surge_charge=0
    if (df['Demand_class']=='Higher_demand') & (df['Supply_class']=='Lower_supply'):
        SD_surge_charge = df['base_cost'] * (demand_hike * df['demand_supply_factor'])
    return SD_surge_charge

df['S/D_surge_charge'] = df.apply(apply_surge,axis=1)
df.head()

,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride,Demand_Supply_Ratio,Demand_class,Supply_class,base_cost,rider_driver_ratio,demand_supply_factor,S/D_surge_charge
0,90,45,Urban,Silver,13,4.47,Night,Premium,90,284.257273,2.000000,Lower_demand,Higher_supply,315.0,2.000000,1.000000,0.000000
1,58,39,Suburban,Silver,72,4.06,Evening,Economy,43,173.874753,1.487179,Lower_demand,Higher_supply,150.5,1.487179,0.487179,0.000000
2,42,31,Rural,Silver,0,3.99,Afternoon,Premium,76,329.795469,1.354839,Lower_demand,Higher_supply,266.0,1.354839,0.354839,0.000000
3,89,28,Rural,Regular,67,4.31,Afternoon,Premium,134,470.201232,3.178571,Higher_demand,Lower_supply,469.0,3.178571,2.178571,357.612500
4,78,22,Rural,Regular,74,3.77,Afternoon,Economy,149,579.681422,3.545455,Higher_demand,Lower_supply,521.5,3.545455,2.545455,464.609091


## Conditional Surge based on Vehical_Type and Time_of_booking && Location_Category Condition


In [6]:

def cal_surge_charge(df):
    surge_charge = 0
    if df['Vehicle_Type'] == 'Premium':
        if (df['Location_Category'] in ('Urban', 'Suburban')) & (df['Time_of_Booking'] in ('Evening', 'Night')):
            surge_charge = df['base_cost'] * 0.05 + df['base_cost'] * 0.02
    else:
        if (df['Location_Category'] in ('Urban', 'Suburban')) & (df['Time_of_Booking'] in ('Evening', 'Night')):
            surge_charge = df['base_cost'] * 0.025 + df['base_cost'] * 0.01
    return surge_charge

df['Surge_charge'] = df.apply(cal_surge_charge, axis=1)


In [7]:
df.sample(10)

,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride,Demand_Supply_Ratio,Demand_class,Supply_class,base_cost,rider_driver_ratio,demand_supply_factor,S/D_surge_charge,Surge_charge
894,49,29,Suburban,Regular,35,4.80,Evening,Premium,160,468.306920,1.689655,Lower_demand,Higher_supply,560.0,1.689655,0.689655,0.000000,39.2000
739,32,6,Rural,Gold,75,4.47,Morning,Premium,154,721.721269,5.333333,Higher_demand,Lower_supply,539.0,5.333333,4.333333,817.483333,0.0000
497,64,7,Suburban,Regular,92,4.58,Evening,Premium,16,114.699202,9.142857,Higher_demand,Lower_supply,56.0,9.142857,6.000000,117.600000,3.9200
284,93,67,Urban,Silver,21,4.88,Night,Premium,87,417.913141,1.388060,Lower_demand,Higher_supply,304.5,1.388060,0.388060,0.000000,21.3150
367,86,20,Suburban,Silver,51,3.75,Morning,Premium,110,412.257463,4.300000,Higher_demand,Lower_supply,385.0,4.300000,3.300000,444.675000,0.0000
925,73,27,Urban,Regular,55,4.27,Morning,Premium,50,216.060196,2.703704,Higher_demand,Lower_supply,175.0,2.703704,1.703704,104.351852,0.0000
846,38,22,Urban,Gold,60,3.60,Morning,Premium,104,312.173159,1.727273,Lower_demand,Higher_supply,364.0,1.727273,0.727273,0.000000,0.0000
452,57,46,Rural,Regular,42,4.39,Morning,Economy,56,210.631767,1.239130,Lower_demand,Higher_supply,196.0,1.239130,0.239130,0.000000,0.0000
193,61,26,Rural,Gold,21,3.85,Morning,Premium,126,604.273047,2.346154,Higher_demand,Lower_supply,441.0,2.346154,1.346154,207.778846,0.0000
179,48,17,Suburban,Silver,2,4.09,Evening,Economy,85,261.943533,2.823529,Higher_demand,Lower_supply,297.5,2.823529,1.823529,189.875000,10.4125


## Calculating Total cost

In [8]:
df['New_cost']= df['base_cost'] + df['S/D_surge_charge'] + df['Surge_charge']
df.iloc[:,[0,1,9,10,11,12,13,16,17,18]].sample(10)

,Number_of_Riders,Number_of_Drivers,Historical_Cost_of_Ride,Demand_Supply_Ratio,Demand_class,Supply_class,base_cost,S/D_surge_charge,Surge_charge,New_cost
534,55,23,654.676536,2.391304,Higher_demand,Lower_supply,525.0,255.652174,0.00,780.652174
80,42,22,366.151526,1.909091,Lower_demand,Higher_supply,287.0,0.000000,0.00,287.000000
925,73,27,216.060196,2.703704,Higher_demand,Lower_supply,175.0,104.351852,0.00,279.351852
67,76,40,96.035209,1.900000,Lower_demand,Higher_supply,98.0,0.000000,0.00,98.000000
708,50,40,319.110983,1.250000,Lower_demand,Higher_supply,255.5,0.000000,0.00,255.500000
415,94,27,547.981462,3.481481,Higher_demand,Lower_supply,602.0,522.848148,42.14,1166.988148
133,85,36,506.692886,2.361111,Higher_demand,Lower_supply,532.0,253.438889,0.00,785.438889
309,42,8,599.716253,5.250000,Higher_demand,Lower_supply,581.0,864.237500,0.00,1445.237500
55,97,55,584.995871,1.763636,Lower_demand,Higher_supply,525.0,0.000000,36.75,561.750000
759,82,59,284.294421,1.389831,Lower_demand,Higher_supply,234.5,0.000000,0.00,234.500000


## Revenue Before and after

In [9]:
print("Revenue before applying Dynamic_pricing -->",round(sum(df['Historical_Cost_of_Ride']),2))
print("Revenue after applying Dynamic_pricing-->",round(sum(df['New_cost']),2))

Revenue before applying Dynamic_pricing --> 372502.62
Revenue after applying Dynamic_pricing--> 552298.01


In [10]:
diff=sum(df['New_cost'])-sum(df['Historical_Cost_of_Ride'])
print("Diffrenece of Revenue--> ", diff)
print("Revenue Percentage --> ", diff/sum(df['Historical_Cost_of_Ride'])*100)

Diffrenece of Revenue-->  179795.39090132003
Revenue Percentage -->  48.26687911203326


## Conclusion
* ### Diffrenece of Revenue-->  179795.39
* ### Revenue Percentage -->  48.26

In [11]:
filter=df['Demand_Supply_Ratio']>10
df[filter].head(10)

,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride,Demand_Supply_Ratio,Demand_class,Supply_class,base_cost,rider_driver_ratio,demand_supply_factor,S/D_surge_charge,Surge_charge,New_cost
49,67,6,Rural,Gold,15,3.53,Night,Economy,123,420.623911,11.166667,Higher_demand,Lower_supply,430.5,11.166667,6.0,904.05,0.0000,1334.5500
88,66,6,Rural,Regular,23,4.20,Evening,Economy,45,173.157754,11.000000,Higher_demand,Lower_supply,157.5,11.000000,6.0,330.75,0.0000,488.2500
94,95,7,Rural,Gold,40,4.68,Evening,Economy,95,283.466443,13.571429,Higher_demand,Lower_supply,332.5,13.571429,6.0,698.25,0.0000,1030.7500
153,51,5,Urban,Gold,0,4.59,Afternoon,Premium,92,320.857622,10.200000,Higher_demand,Lower_supply,322.0,10.200000,6.0,676.20,0.0000,998.2000
170,76,7,Urban,Gold,76,4.35,Morning,Economy,72,245.893571,10.857143,Higher_demand,Lower_supply,252.0,10.857143,6.0,529.20,0.0000,781.2000
197,75,7,Suburban,Gold,100,4.13,Morning,Economy,134,453.376949,10.714286,Higher_demand,Lower_supply,469.0,10.714286,6.0,984.90,0.0000,1453.9000
216,88,5,Urban,Silver,89,3.59,Night,Economy,27,70.203803,17.600000,Higher_demand,Lower_supply,94.5,17.600000,6.0,198.45,3.3075,296.2575
218,65,5,Rural,Silver,24,3.54,Night,Economy,119,301.403927,13.000000,Higher_demand,Lower_supply,416.5,13.000000,6.0,874.65,0.0000,1291.1500
232,87,5,Urban,Silver,59,4.32,Night,Economy,42,151.359301,17.400000,Higher_demand,Lower_supply,147.0,17.400000,6.0,308.70,5.1450,460.8450
250,97,7,Urban,Silver,22,3.74,Afternoon,Premium,147,441.746701,13.857143,Higher_demand,Lower_supply,514.5,13.857143,6.0,1080.45,0.0000,1594.9500


# Data Spliting and Model training

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


In [13]:
x=df[['Demand_Supply_Ratio','Location_Category','Time_of_Booking','Vehicle_Type','Expected_Ride_Duration']] #Train column
y=df['New_cost'] #Target column

In [14]:
X_train_val, X_test, y_train_val, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

In [30]:
X_train

,Demand_Supply_Ratio,Location_Category,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration
876,1.126437,Suburban,Afternoon,Economy,13
326,1.700000,Urban,Morning,Premium,158
381,9.142857,Rural,Evening,Economy,64
853,1.282051,Urban,Morning,Premium,135
311,2.266667,Urban,Night,Premium,134
...,...,...,...,...,...
118,4.043478,Suburban,Evening,Premium,125
334,3.700000,Urban,Afternoon,Economy,44
409,3.142857,Urban,Morning,Premium,98
225,7.571429,Urban,Night,Premium,10


In [16]:
y_train

876     45.500000
326    553.000000
381    694.400000
853    472.500000
311    501.830000
          ...    
118    934.157609
334    299.530000
409    600.250000
225    110.950000
482    472.546974
Name: New_cost, Length: 600, dtype: float64

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [32]:
ohe = OneHotEncoder()
ohe.fit(x[['Location_Category','Time_of_Booking','Vehicle_Type']])

OneHotEncoder()

In [33]:
cat=ohe.categories_

In [34]:
column_trans = make_column_transformer((OneHotEncoder(categories=cat),
                                        ['Location_Category','Time_of_Booking','Vehicle_Type']),
                                        remainder='passthrough')                                        

In [35]:
lr=LinearRegression()

In [36]:
pipe=make_pipeline(column_trans,lr)

In [37]:
pipe.fit(X_train,y_train)

c:\Users\sanoj\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Rural', 'Suburban', 'Urban'], dtype=object),
                                                                            array(['Afternoon', 'Evening', 'Morning', 'Night'], dtype=object),
                                                                            array(['Economy', 'Premium'], dtype=object)]),
                                                  ['Location_Category',
                                                   'Time_of_Booking',
                                                   'Vehicle_Type'])])),
                ('linearregression', LinearRegression())])

In [24]:
y_pred=pipe.predict(X_test)
y_pred

array([ 725.49985318,  508.1060693 ,  988.29705641,  393.26549241,
        318.97656193,  531.98143912,  423.53447011,  119.30788357,
        693.75509236,  169.27977397,  398.63716732,  210.36750737,
        193.75497913,  889.54894577,  497.90395848,  500.94705098,
        880.32441075,  625.99665575,  116.33528796,  451.19424188,
        614.82082844,  291.05957571,  297.44714832,  465.99765218,
        291.73430176,  914.86394215,   99.02837815, 1400.26007905,
        203.81191271, 1115.29983168,  377.38545155,   24.37875791,
        543.65000518,  111.74637726,  528.84416851,  374.28285041,
        121.44603573,  297.59227745,  515.23845114,  462.21432185,
        876.56109658,  815.45941023, 1008.81681601,  949.42843104,
       1314.13789469,  516.99502246,  422.34182859,  973.97749499,
        335.00744433,  556.77384055,  450.69908446,   84.22266332,
        662.45194721,  249.6198604 ,  134.94326023,  451.84849959,
        422.47184824, 1032.20481351,  875.95829501,  721.90047

# Model Evaluation & Check prediction 

In [45]:
from sklearn.metrics import mean_absolute_percentage_error,mean_squared_error

### Model evaluation on Validation data

In [53]:
y_val_pred = pipe.predict(X_val)
mape = mean_absolute_percentage_error(y_val,y_val_pred)
print("Error of Linear Regression Model = %.2f"%(mape*100),'%')
print("Accuracy of Linear Regression Model = %.2f"%((1 - mape)*100),'%')

Error of Linear Regression Model = 29.48 %
Accuracy of Linear Regression Model = 70.52 %


In [41]:
r2 = r2_score(y_val,y_val_pred)
print("R2 score of Linear Regression = %.2f"%(r2))

R2 score of Linear Regression = 0.81


### Model evaluation on Test Data

In [55]:
y_test_pred = pipe.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)
print("Mean square error :", test_mse)
print("R2 score of Linear Regression on test data:", test_r2*100)

Mean square error : 25804.010772131278
R2 score of Linear Regression on test data: 83.08779931645755


In [42]:
pipe.predict(pd.DataFrame([['2.0','Urban','Night','Premium','90']],columns=['Demand_Supply_Ratio','Location_Category','Time_of_Booking','Vehicle_Type','Expected_Ride_Duration']))

array([411.07015495])

In [65]:
results_df = pd.DataFrame(X_test, columns=x.columns)
results_df['Actual'] = y_test
results_df['Predicted'] = y_pred
results_df['Error'] = y_pred-y_test
results_df.to_csv("test_results.csv", index=False)

In [74]:
df2=pd.read_csv('test_results.csv')
df2.iloc[:,[5,6,7]]
df2.head(10)

,Demand_Supply_Ratio,Location_Category,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Actual,Predicted,Error
0,4.750000,Urban,Night,Economy,114,936.652500,725.499853,-211.152647
1,4.611111,Urban,Morning,Economy,81,641.812500,508.106069,-133.706431
2,3.142857,Urban,Evening,Premium,180,1146.600000,988.297056,-158.302944
3,2.230769,Suburban,Evening,Premium,87,325.815000,393.265492,67.450492
4,6.500000,Suburban,Night,Premium,19,199.167500,318.976562,119.809062
5,2.956522,Suburban,Morning,Economy,105,619.157609,531.981439,-87.176170
6,5.333333,Urban,Afternoon,Economy,53,466.841667,423.534470,-43.307197
7,1.555556,Suburban,Afternoon,Premium,43,150.500000,119.307884,-31.192116
8,3.857143,Urban,Morning,Premium,120,840.000000,693.755092,-146.244908
9,2.935484,Urban,Night,Economy,39,233.745242,169.279774,-64.465468


In [67]:
df2['Actual'].sum()

114642.71250925891

In [68]:
df2['Predicted'].sum()

110765.39342615806

In [71]:
diff=df2['Actual'].sum()-df2['Predicted'].sum()
diff

3877.319083100854

In [77]:
df2['Error'].sum()

-3877.3190831008574